In [ ]:
# -*- coding: utf-8 -*-
"""RL_Transformers.ipynb

In [ ]:
Automatically generated by Colaboratory.

In [ ]:
Original file is located at
    https://colab.research.google.com/drive/1_V_Rl_Row_jpku091J0dJ2-lAEYtTmMp
"""

Commented out IPython magic to ensure Python compatibility.<br>
title install { form-width: "200px" }<br>
%cd /content/drive/MyDrive/Thesis/RL_scbert_bart_xsum<br>
pip install -r requirements.txt<br>
%cd /content/drive/MyDrive/Thesis/RL_scbert_bart_xsum/trl/trl<br>
pip install transformers

In [ ]:
from transformers import AutoModelForSeq2SeqLM, BartTokenizer, BartModel, BartForConditionalGeneration, BartConfig, GPT2Config,GPT2LMHeadModel
from transformers import top_k_top_p_filtering, GPT2Model
from torch import nn
from torch.nn import Identity
import torch.nn.functional as F
import torch

imports

In [ ]:
import torch
from transformers import GPT2Tokenizer, AutoModel, BartTokenizer,AutoTokenizer
import sys
sys.path.append('D:\\Thesis\\RL_scbert_bart_xsum\\trl\\trl')
from gpt2 import *

In [ ]:
import gpt2
from bart_xsum import *
from ppo import *
from transformers import BartPretrainedModel

In [ ]:
import numpy as np
def setup_seed(seed):
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  torch.backends.cudnn.deteministic = True
setup_seed(57)


# **Load Model**
<br>
#@title init reward model { form-width: "10%" }<br>
from transformers import BertModel,BertPreTrainedModel<br>
import torch.nn as nn<br>
class BertRegresser(BertPreTrainedModel):<br>
    def __init__(self, config):<br>
        super().__init__(config)<br>
        self.bert = BertModel(config)<br>
        #The output layer that takes the [CLS] representation and gives an output<br>
        self.regressor = nn.Sequential(<br>
            nn.Dropout(0.5),<br>
            nn.Linear(768, 1))<br>
    def forward(self, input_ids, attention_mask):<br>
        #Feed the input to Bert model to obtain contextualized representations<br>
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)<br>
        #Obtain the representations of [CLS] heads<br>
        logits = outputs[1]<br>
        output = self.regressor(logits)<br>
        <br>
        return output<br>
#@title load model, reference model and reward model { form-width: "10%" }<br>
import os<br>
config = BartConfig('facebook/bart-large-xsum', output_hidden_states=True)<br>
model_name = 'D:\\Thesis\\RL_scbert_bart_xsum\\bart_large_xsum'<br>
#load preptrained model<br>
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, output_hidden_states=True).to('cuda')<br>
#load reference model<br>
ref_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, output_hidden_states=True).to('cuda')<br>
#load reward model<br>
reward_model = BertRegresser.from_pretrained('allenai/scibert_scivocab_uncased')<br>
model_state, optimizer_state = torch.load('D:\\Thesis\\Models\\reward_model\\checkpoint')<br>
reward_model.load_state_dict(model_state)<br>
reward_model.to('cuda')<br>
#load Valuehead<br>
hmodel = ValueHead(config).to('cuda')<br>
#@title load tokenizer<br>
from transformers import AutoTokenizer<br>
#load reward tokenizer<br>
reward_tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')<br>
#load model tokenizer<br>
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-xsum')<br>
 **Load Data (Final Model is trained with "gen_title_score_pairs_bestone")**


title Load annotated data

In [ ]:
import pandas as pd
anno_sample = pd.read_json('D:\\Thesis\\output\\annotated_json\\dataset_140samples.json')
def creat_tupel(sample_np):
  ab = [sample_np[0]]*6
  ht = [sample_np[1]]*6
  indices = np.array([2, 5, 8, 11, 14, 17])
  sysms = list(sample_np[2].keys())
  sysms = ['original'] + sysms
  gents = list(sample_np[2].values())
  gents = [sample_np[1]] + gents
  gents_scores = list(sample_np[3].values())
  gents_scores = [gents_scores[i] for i in indices]
  max_idx = gents_scores.index(max(gents_scores))
  res = np.transpose(np.array([ab, ht, gents, gents_scores, sysms]))
  return res[max_idx,:].reshape(1,5), res
anno_sample_np =anno_sample.to_numpy()
res = []
res1 = []
for row in anno_sample_np:
  r1, r2 = creat_tupel(row)
  res += r1.tolist()
  res1 += r2.tolist()
gen_title_score_pairs = np.array(res1)
gen_title_score_pairs_bestone = np.array(res)


# **3 Datasettings (only for analysis in default RL, final model using cross learning with all sample witgh best title)**
<br>
#@title pick bad annotated data or from bart-xsum<br>
gen_title_score_pairs_bad = []<br>
scores = gen_title_score_pairs_bestone[:,-2]<br>
scores = np.array([float(s) for s in scores])<br>
for row in gen_title_score_pairs_bestone:<br>
  if float(row[-2]) <= np.mean(scores):<br>
    gen_title_score_pairs_bad.append(row)<br>
gen_title_score_pairs_bad = np.array(gen_title_score_pairs_bad)<br>
#@title pick annotated data from bart-xsum<br>
gen_title_score_pairs_xsum = []<br>
for row in gen_title_score_pairs:<br>
  if row[-1] == 'bart_xsum':<br>
    gen_title_score_pairs_xsum.append(row)<br>
gen_title_score_pairs_xsum = np.array(gen_title_score_pairs_xsum)<br>
#@title pick good annotated data or from bart-xsum<br>
gen_title_score_pairs_good = []<br>
scores = gen_title_score_pairs_bestone[:,-2]<br>
scores = np.array([float(s) for s in scores])<br>
for row in gen_title_score_pairs_bestone:<br>
  if float(row[-2]) >= np.mean(scores) or row[-1] == 'bart_xsum':<br>
    gen_title_score_pairs_good.append(row)<br>
gen_title_score_pairs_good = np.array(gen_title_score_pairs_good)<br>
 **2 Train Mode (Cross learning/ Default RL)**


title cross learning trainer { form-width: "10%" }

In [ ]:
gen_kwargs = {
    "min_length":-1,
    "top_k": 2,
    "top_p": 0.8,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    #"length_penalty" : -20.0,
    #"num_return_sequences" : 5,
    #"repetition_penalty" : 1.5,
}
def ACT_step(title_score_pairs, start_ids, stop_ids, tokenizer, model, ppo_trainer):
  res = []
  for i in range(start_ids, stop_ids):
    row = title_score_pairs[i]
    ref_response_txt = row[2]
    #model_name = row[4]
    original_title = row[1]
    query_txt = row[0]
    query_tensor = tokenizer(query_txt, return_tensors="pt").to('cuda')
    #act step
    ref_response_tensor = tokenizer(ref_response_txt, return_tensors='pt').to('cuda')
     # define a reward for response
    t = '[CLS]' + query_txt + '[SEP]' + ref_response_txt
    ref_reward_encode = reward_tokenizer(t, return_tensors='pt').to('cuda')
    #reward_input = reward_encode.input_ids
    #reward_att = reward_encode.attention_mask
    ref_reward = reward_model(ref_reward_encode['input_ids'], ref_reward_encode['attention_mask']).squeeze(-1)
    ref_reward = torch.tensor([ref_reward.item()]).to('cuda')
    #reward = torch.tensor(float(row[3])).to('cuda')
    # train model with ppo
    train_stats = ppo_trainer.step(query_tensor["input_ids"], ref_response_tensor["input_ids"], ref_reward)

    # get model response
    '''response_tensor = model.generate(
                input_ids = query_tensor["input_ids"],
                attention_mask = query_tensor["attention_mask"],
                max_length = 30,
                num_beams = 5,
                num_return_sequences = 1,
                repetition_penalty=2.0, 
                length_penalty=10.0,
                early_stopping = True,
                ).to('cuda')'''
    response_tensor = model.generate(input_ids = query_tensor["input_ids"],
                attention_mask = query_tensor["attention_mask"], max_new_tokens=30, **gen_kwargs)[-30:].to('cuda')
    response_txt = tokenizer.batch_decode(response_tensor, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    #test ref model response(generated title from fine-tunde bart-xsum)
    

    # define a reward for response
    t = '[CLS]' + query_txt + '[SEP]' + response_txt
    reward_encode = reward_tokenizer(t, return_tensors='pt').to('cuda')
    #reward_input = reward_encode.input_ids
    #reward_att = reward_encode.attention_mask
    reward = reward_model(reward_encode['input_ids'], reward_encode['attention_mask']).squeeze(-1)
    reward = torch.tensor([reward.item()]).to('cuda')

    # train model with ppo
    train_stats = ppo_trainer.step(query_tensor["input_ids"], response_tensor, reward)

    #generate title with rewarded model
    '''new_model_response_ids = model.generate(
                input_ids = query_tensor["input_ids"],
                attention_mask = query_tensor["attention_mask"],
                max_length = 30,
                num_beams = 5,
                num_return_sequences = 1,
                repetition_penalty=2.0, 
                length_penalty=10.0,
                early_stopping = True,
                )'''
    new_model_response_ids = model.generate(input_ids = query_tensor["input_ids"],
                attention_mask = query_tensor["attention_mask"], max_new_tokens=30, **gen_kwargs)[-30:].to('cuda')
    new_model_response_ids = new_model_response_ids.cpu().data.numpy()
    new_model_response_txt = tokenizer.batch_decode(new_model_response_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    res.append([response_txt, new_model_response_txt])
    if i % 10 == 0:
      print('Original title: ', original_title)
      print('Generated reference title: ', ref_response_txt)
      print('RL-rewarded-after-step bart_xsum generated title: ', new_model_response_txt)
      print('- - - -'*20 + '>')
  return res

title default RL trainer { form-width: "10%" }

In [ ]:
gen_kwargs = {
    "min_length":-1,
    "top_k": 2,
    "top_p": 0.8,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    #"length_penalty" : 4.0,
    #"repetition_penalty" : 1.5,
}
def RL_steps(title_score_pairs, start_ids, stop_ids, tokenizer, model, ppo_trainer):
  res = []
  for i in range(start_ids, stop_ids):
    row = title_score_pairs[i]
    #model_name = row[4]
    original_title = row[1]
    query_txt = row[0]
    query_tensor = tokenizer(query_txt, return_tensors="pt").to('cuda')
    # get model response
    '''response_tensor = model.generate(
                input_ids = query_tensor["input_ids"],
                attention_mask = query_tensor["attention_mask"],
                max_length = 30,
                num_beams = 5,
                num_return_sequences = 1,
                repetition_penalty=2.0, 
                length_penalty=10.0,
                early_stopping = True,
                ).to('cuda')'''
                
    response_tensor = model.generate(input_ids = query_tensor["input_ids"],
                attention_mask = query_tensor["attention_mask"], max_new_tokens=30, **gen_kwargs)[-30:].to('cuda')
    response_txt = tokenizer.batch_decode(response_tensor, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    #test ref model response(generated title from fine-tunde bart-xsum)
    #ref_response_txt = row[2]

    # define a reward for response
    t = '[CLS]' + query_txt + '[SEP]' + response_txt
    reward_encode = reward_tokenizer(t, return_tensors='pt').to('cuda')
    #reward_input = reward_encode.input_ids
    #reward_att = reward_encode.attention_mask
    reward = reward_model(reward_encode['input_ids'], reward_encode['attention_mask']).squeeze(-1)
    #reward = row[2].item()
    reward = torch.tensor([reward.item()]).to('cuda')

    # train model with ppo
    train_stats = ppo_trainer.step(query_tensor["input_ids"], response_tensor, reward)

    #generate title with rewarded model
    new_model_response_ids = model.generate(
                input_ids = query_tensor["input_ids"],
                attention_mask = query_tensor["attention_mask"],
                max_length = 30,
                num_beams = 5,
                num_return_sequences = 1,
                repetition_penalty=2.0, 
                length_penalty=10.0,
                early_stopping = True,
                )
    new_model_response_ids = new_model_response_ids.cpu().data.numpy()
    new_model_response_txt = tokenizer.batch_decode(new_model_response_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    res.append([response_txt, new_model_response_txt])
    if i % 6 == 0:
      print('Original title: ', original_title)
      #print('Generated reference title: ', ref_response_txt)
      print('RL-rewarded-after-step bart_xsum generated title: ', new_model_response_txt)
      print('- - - -'*20 + '>')
  return res

title test setup


<br>
df1 = pd.read_csv('/content/drive/MyDrive/Thesis/RL_scbert_bart_xsum/output/output_111_annotated_from_bart_xsum_lowLR.csv', index_col=0)<br>
df2 = pd.read_csv('/content/drive/MyDrive/Thesis/RL_scbert_bart_xsum/output/output_111_annotated_from_bart_xsum_midLrEp.csv', index_col=0)<br>
df3 = pd.read_csv('/content/drive/MyDrive/Thesis/RL_scbert_bart_xsum/output/output_111_annotated_from_bart_xsum_verylowLR.csv', index_col=0)<br>
df4 = pd.read_csv('/content/drive/MyDrive/Thesis/RL_scbert_bart_xsum/output/output_111_annotated_from_bart_xsum_midLrEp_good_scores.csv', index_col=0)<br>
ab = df1[['abstract']].to_numpy().squeeze()<br>
ot = df1[['original title']].to_numpy().squeeze()<br>
gt = df1[['generated title before RL']].to_numpy().squeeze()<br>
t1 = df1[['generated title after RL']].to_numpy().squeeze()<br>
t2 = df2[['title-xsum-reward']].to_numpy().squeeze()<br>
t3 = df3[['generated title after RL']].to_numpy().squeeze()<br>
t4 = df1[['generated title after RL']].to_numpy().squeeze()<br>
ttt = np.array([ab,ot,gt,t1,t2,t3,t4]).transpose()<br>
df = pd.DataFrame(ttt)<br>
df.columns =['abstracz','human title', 'before RL', 'low LR', 'mid LR', 'very low LR', 'mid LR goog only']<br>
df.to_csv('/content/drive/MyDrive/Thesis/RL_scbert_bart_xsum/output/output_111_dif_setup.csv')<br>



# **Train Settings and Do Train**
<br>
from threading import active_count<br>
#@title initialize PPOtrainer<br>
# initialize trainer<br>
#ppo_config = {'batch_size': 1, 'forward_batch_size': 1}<br>
ppo_config = {<br>
        "lr": 2e-6,#1.41e-5,#6e-7,#"lr": 3e-6,#,<br>
        "adap_kl_ctrl": True,<br>
        "init_kl_coef":0.2,<br>
        "target": 6,<br>
        "horizon":10000,<br>
        "gamma":1,<br>
        "lam":0.95,<br>
        "cliprange": .2,<br>
        "cliprange_value":.2,<br>
        "vf_coef":.1,<br>
        "batch_size": 1,<br>
        "forward_batch_size": 1,<br>
        "ppo_epochs": 3,<br>
    }<br>
ppo_trainer = PPOTrainer(model, ref_model, hmodel, **ppo_config)<br>
res = ACT_step(gen_title_score_pairs_bestone, 0, len(gen_title_score_pairs_bestone), tokenizer, model, ppo_trainer)<br>
#res = RL_steps(gen_title_score_pairs_bad, 0, len(gen_title_score_pairs_bad), tokenizer, model, ppo_trainer)<br>
 **Debug**


In [ ]:
query_txt = "This bachelor thesis explores the generation of title based on a given abstract using neural language model. Recently, neural language models have been used in many scenarios with practical applications. For example, in scientific writing, automatic summary generation from long texts is used to assist in the reading and selection of relevant scientific articles. Title is an important part of scientific article, but the title generation using neural language and optimization for neural language model based on human preferences are less studied. This thesis addresses this gap and presents an optimized model based on state-of-the-art pre-trained neural language model which generate human-preferred titles from a given abstract. The model is fine-tuned on datasets of scientific article and optimized from human preferences using the novel learning perspective in reinforcement learning environment. The result shows that, the neural language model have powerful capabilities on the abstract-to-title task and the reinforcement learning approach is effective in scalable learning of neural language model."
 
query_tensor = tokenizer(query_txt, return_tensors="pt").to('cuda')

In [ ]:
ref_tensor=model.generate(input_ids = query_tensor["input_ids"], 
                          attention_mask = query_tensor["attention_mask"], 
                          max_new_tokens=30, 
                          **gen_kwargs)[-30:].to('cuda')

In [ ]:
ref_tensor = model.generate(
              input_ids = query_tensor["input_ids"],
              attention_mask = query_tensor["attention_mask"],
              max_length = 30,
              num_beams = 5,
              num_return_sequences = 1,
              repetition_penalty=2.0, 
              length_penalty=10.0,
              early_stopping = True,
              ).to('cuda')

In [ ]:
ref_txt = tokenizer.batch_decode(ref_tensor, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]


# **Generation**
<br>
#@title write out file { form-width: "10%" }<br>
outres = []<br>
for row in gen_title_score_pairs_bestone:<br>
  ab = row[0]<br>
  ht = row[1]<br>
  ogt = row[2]<br>
  query_txt = ab<br>
  query_tensor = tokenizer(query_txt, return_tensors="pt").to('cuda')<br>
  #get ref model response<br>

ref_tensor = ref_model.generate(

In [ ]:
              input_ids = query_tensor["input_ids"],
              attention_mask = query_tensor["attention_mask"],
              max_length = 30,
              num_beams = 5,
              num_return_sequences = 1,
              repetition_penalty=2.0, 
              length_penalty=10.0,
              early_stopping = True,
              ).to('cuda')'''
  ref_tensor=ref_model.generate(input_ids = query_tensor["input_ids"], 
                          attention_mask = query_tensor["attention_mask"], 
                          max_new_tokens=30, 
                          **gen_kwargs)[-30:].to('cuda')
  ref_txt = tokenizer.batch_decode(ref_tensor, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
  # get model response
  '''response_tensor = model.generate(
              input_ids = query_tensor["input_ids"],
              attention_mask = query_tensor["attention_mask"],
              max_length = 30,
              num_beams = 5,
              num_return_sequences = 1,
              repetition_penalty=2.0, 
              length_penalty=10.0,
              early_stopping = True,
              ).to('cuda')'''
  response_tensor = model.generate(input_ids = query_tensor["input_ids"], attention_mask = query_tensor["attention_mask"], max_new_tokens=30, **gen_kwargs)[-30:].to('cuda')

In [ ]:
  response_txt = tokenizer.batch_decode(response_tensor, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
  outres.append([ab, ht,ogt, ref_txt, response_txt])
import pandas as pd

In [ ]:
df = pd.DataFrame(outres)

In [ ]:
df.columns = ['abstract', 'original title','best title' , 'generated title before RL', 'generated title after RL']
path='D:\Thesis\RL_scbert_bart_xsum\output\act_'+ "{:.9f}".format(ppo_config['lr'])+'_'+str(ppo_config['ppo_epochs'])+'.csv'

f.to_csv(path)

In [ ]:
df = df.drop(['original title'], axis=1)
df.columns = ['abstract', 'best title', 'title-xsum', 'title-xsum-reward']
df_np = df.to_numpy()
n_np = []
for row in df_np:
  ab = row[0]
  ts = row[1:]
  #ts[0] = ts[0]
  #ts[0] = ts[0]
  #ts = np.random.permutation(ts)
  n_np.append([ab, ts[1], ts[2]])
n_np = np.array(n_np)
se = pd.DataFrame(n_np[:30])


# **Analysis**
<br>
import pandas as pd<br>
#@title load bartscore scorer<br>
# %cd /content/drive/MyDrive/Thesis/BARTScore<br>
import sys<br>
sys.path.append('D:\\Thesis\\BARTScore-main')<br>
from bart_score import BARTScorer<br>
bart_scorer = BARTScorer(device='cuda:0',checkpoint='D:\\Thesis\\BARTScore-main\\bart_xsum')<br>
# Commented out IPython magic to ensure Python compatibility.<br>
#@title load moverscore scorer<br>
# %cd /content/drive/MyDrive/Thesis/emnlp19-moverscore-master<br>
ys.path.append('D:\\Thesis\\emnlp19-moverscore-master')

In [ ]:
from moverscore_v2 import get_idf_dict, word_mover_score
from typing import List, Union, Iterable
from collections import defaultdict
import numpy as np
def sentence_score(hypothesis: str, references: List[str], trace=0):
    
    idf_dict_hyp = defaultdict(lambda: 1.)
    idf_dict_ref = defaultdict(lambda: 1.)
    
    hypothesis = [hypothesis] * len(references)
    
    sentence_score = 0 
    scores = word_mover_score(references, hypothesis, idf_dict_ref, idf_dict_hyp, stop_words=[], n_gram=1, remove_subwords=False)
    
    sentence_score = np.mean(scores)
    
    if trace > 0:
        print(hypothesis, references, sentence_score)
            
    return sentence_score
'''
# Commented out IPython magic to ensure Python compatibility.
#@title load bertscore scorer

%cd /content/drive/MyDrive/Thesis/bert_score

In [ ]:
import torch
sys.path.append('D:\\Thesis\\bert_score-master')
from bert_score import score

In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
df = pd.read_csv('D:\\Thesis\\RL_scbert_bart_xsum\\output\\output_111_annotated_from_bart_xsum_act_bad_final0.000002000_3.csv', index_col=0)[:30]

In [ ]:
abs = df['abstract'].to_list()
ots = df['original title'].to_list()
bts = df['best title'].to_list()
ogts = df['generated title before RL'].to_list()
rlts = df['generated title after RL'].to_list()

In [ ]:
scores11 = bart_scorer.score(abs, ots, batch_size=1)
scores21 = bart_scorer.score(abs, ogts, batch_size=1)
scores31 = bart_scorer.score(abs, rlts, batch_size=1)
'''
scores12 = [sentence_score(ot, [ab]) for ot,ab in zip(ots, abs)]
scores22 = [sentence_score(ogt, [ab]) for ogt,ab in zip(ogts, abs)]
scores32 = [sentence_score(rlt, [ab]) for rlt,ab in zip(rlts, abs)]'''

In [ ]:
p,r,score13 = score(ots, abs, lang="en")
p,r,score23 = score(ogts, abs, lang="en")
p,r,score33 = score(rlts, abs , lang="en")

ARTScore

In [ ]:
print('abs_ots: ', sum(scores11)/len(scores11))
print('abs_ogts: ', sum(scores21)/len(scores21))
print('abs_rlts: ', sum(scores31)/len(scores31))
import numpy as np
#BertScore
print('abs_ots: ', np.array(score13.tolist()).mean())
print('abs_ogts: ', np.array(score23.tolist()).mean())
print('abs_rlts: ', np.array(score33.tolist()).mean())

overScore


print('abs_ots: ', sum(scores12)/len(scores12))<br>
print('abs_ogts: ', sum(scores22)/len(scores22))<br>
int('abs_rlts: ', sum(scores32)/len(scores32))


In [ ]:
r1 = [np.exp(-0.654991332689921), 0.8518536269664765, 0.5175741747308048]

In [ ]:
r2 = [np.exp(-0.6468217780192693), 0.8508123060067495, 0.5174440166876286]

3 = [np.exp(-0.6468217780192693), 0.5174440166876286, 0.8508123060067495]

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import numpy as np
normalized_metrics = normalize(np.array([r1, r2]), axis=0, norm='l1')
normalized_metrics

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import numpy as np
normalized_metrics = normalize(np.array([r1, r2]), axis=0, norm='l1')
normalized_metrics